In [382]:
import numpy as np
import networkx as nx

In [380]:
def do_predication(arg, pred, topn=20, k=5):
    # take argument(A) and predicate(P), to compute the predication P(A)
    # topn: how many neighboring words to predicate to use
    # k: how many neighboring words for predication to use
    
    idx_to_word = {0:arg, 1:pred}
    
    # topn similar words to predicate
    similar_to_pred = model.most_similar([pred], topn=topn)
    
    # create graph for computing spreading activation on predicate and argument
    MetaphorNet = nx.Graph()
    
    MetaphorNet.add_nodes_from([arg, pred])
    MetaphorNet.add_edge(pred, pred, weight=1)
    MetaphorNet.add_edge(arg, arg, weight=1)
    arg_pred_weight = cos(model[arg], model[pred])
    MetaphorNet.add_edge(arg, pred, weight=arg_pred_weight)

    inhibitory_weight = arg_pred_weight
    for word, cosine in similar_to_pred:
        idx_to_word[len(idx_to_word)] = word
        MetaphorNet.add_node(word)
        MetaphorNet.add_edge(word, word, weight=1)
        MetaphorNet.add_edge(pred, word, weight=cosine)

        cos_arg = cos(model[word], model[arg])
        MetaphorNet.add_edge(arg, word, weight=cos_arg)
        inhibitory_weight += cosine + cos_arg
        
    # calculate inhibitory weight for negative links
    n_edges = 3
    for i in range(3, len(similar_to_pred)):
        n_edges += i
    
    inhibitory_weight = -(inhibitory_weight / n_edges)
    
    for word, cosine in similar_to_pred:
        for word2, cosine2 in similar_to_pred:
            if word != word2:
                MetaphorNet.add_edge(word, word2, weight=inhibitory_weight)
            
    metaphornet = nx.adjacency_matrix(MetaphorNet).todense()
    
    # update the state of the network
    metaphornet = metaphornet @ metaphornet
    for i in range(3):
        metaphornet = metaphornet @ metaphornet
        for j in range(metaphornet.shape[0]):
            metaphornet[j] = metaphornet[j] / np.max(metaphornet[j])
            
    # take topk words to predicate after spreading activation
    topk = np.argsort(cosine_similarity(metaphornet[1], metaphornet))[-1][::-1][:k]
    
    # use topk words to calculate predication
    predicated = 0
    for idx in topk:
#         print(idx_to_word[idx])
        word = idx_to_word[idx]
        predicated += model[word]
        
    for i in range(len(topk)):
        predicated[i] /= len(topk)
        
    return predicated

In [381]:
# the example in Kintsch (2000), comparing 'The horse ran' and 'The color ran'
predicated_horse = do_predication('horse', 'ran', topn=20, k=5)
predicated_color = do_predication('color', 'ran', topn=20, k=5)

# result in the paper reports .75, .29, .01, .11
print(cos(predicated_horse, model['gallop']), cos(predicated_color, model['gallop']), cos(predicated_horse, model['dissolve']), cos(predicated_color, model['dissolve']))

0.4739087 0.3530702 0.17367178 0.22149065


In [387]:
# the example in Kintsch (2000), collapsed
predicated_bridge = do_predication('bridge', 'collapsed', topn=20, k=5)
predicated_plans = do_predication('plans', 'collapsed', topn=20, k=5)
predicated_runner = do_predication('runner', 'collapsed', topn=20, k=5)

break_down = model['breakdown']
failure = model['failure']
race = model['race']

# result in the paper reports 1, 2, 3 highest respectively
print(cos(predicated_bridge, break_down), cos(predicated_plans, break_down), cos(predicated_runner, break_down))
print(cos(predicated_bridge, failure), cos(predicated_plans, failure), cos(predicated_runner, failure))
print(cos(predicated_bridge, race), cos(predicated_plans, race), cos(predicated_runner, race))

0.38564736 0.4054022 0.3871393
0.42505452 0.4764815 0.4277407
0.12003168 0.136017 0.17874005
